In [105]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [106]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [107]:
from pyspark.sql.types import IntegerType,StringType,StructField,StructType,DateType,DoubleType

schema = StructType([StructField("Index", IntegerType(), True),
	StructField("Date", DateType(), True),
	StructField("Time", StringType(), True),
	StructField("Total_Precipitation", DoubleType(), True),
	StructField("Atmospheric_Pressure", DoubleType(), True),
	StructField("Maximum_Atmospheric_Pressure", DoubleType(), True),
	StructField("Minimum_Atmospheric_Pressure", DoubleType(), True),
	StructField("Solar_Radiation", DoubleType(), True),
	StructField("Air_Temperature", DoubleType(), True),
	StructField("Dew_Point_Temperature", DoubleType(), True),
	StructField("Maximum_Temperature_For_The_Last_Hour", DoubleType(), True),
	StructField("Minimum_Temperature_For_The_Last_Hour", DoubleType(), True),
	StructField("Maximum_Air_Pressure_Dew_Point_Temperature_For_The_Last_Hour", DoubleType(), True),
	StructField("Minimum_Air_Pressure_Dew_Point_Temperature_For_The_Last_Hour", DoubleType(), True),
	StructField("Maximum_Relative_Humid_Temperature_For_The_Last_Hour", DoubleType(), True),	
	StructField("Minimum_Relative_Humid_Temperature_For_The_Last_Hour", DoubleType(), True),
	StructField("Air_Relative_Humid_Temperature_For_The_Last_Hour", DoubleType(), True),
	StructField("Wind_Direction_Radius_Degree", DoubleType(), True),
	StructField("Wind_Gust_In_Metres_Per_Second", DoubleType(), True),
	StructField("Wind_speed_In_Metres_Per_Second", DoubleType(), True),
    StructField("Region", StringType(), True),
	StructField("State", StringType(), True),
	StructField("Station_Name", StringType(), True),
	StructField("Station_Code", StringType(), True),
	StructField("Latitude", DoubleType(), True),
	StructField("Longitude", DoubleType(), True),
	StructField("Elevation", DoubleType(), True)
	])

In [108]:
file_location = "file:///home/talentum/shared/CDAC_Project_Weather_Anaysis/final_north_weather_data.csv"
df1 = spark.read.csv(file_location,header=True)

In [109]:
df1.show()

+----------+-----+-------------------+--------------------+---------------+-------------------------------------+-------------------------------------+----------------------------------------------------+----------------------------------------------------+-------------------------------+------+-----+------------+------------+-----------+------------+---------+
|      Date| Time|Total_Precipitation|Atmospheric_Pressure|Solar_Radiation|Maximum_Temperature_For_The_Last_Hour|Minimum_Temperature_For_The_Last_Hour|Maximum_Relative_Humid_Temperature_For_The_Last_Hour|Minimum_Relative_Humid_Temperature_For_The_Last_Hour|Wind_speed_In_Metres_Per_Second|Region|State|Station_Name|Station_Code|   Latitude|   Longitude|Elevation|
+----------+-----+-------------------+--------------------+---------------+-------------------------------------+-------------------------------------+----------------------------------------------------+----------------------------------------------------+---------------

In [ ]:
df1.

In [ ]:
new_rows = [Row(Name="David", Age=28), Row(Name="Eva", Age=32)]

# Convert the new rows to a DataFrame
new_df = spark.createDataFrame(new_rows, columns)
final_df = df.union(new_df)

# Show the resulting DataFrame
final_df.show()

In [110]:
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import mean


df1 = df1.withColumn("Total_Precipitation", F.col("Total_Precipitation").cast(DoubleType()))
df1 = df1.withColumn("Atmospheric_Pressure", F.col("Atmospheric_Pressure").cast(DoubleType()))
df1 = df1.withColumn("Solar_Radiation", F.col("Solar_Radiation").cast(DoubleType()))
df1 = df1.withColumn("Maximum_Temperature_For_The_Last_Hour", F.col("Maximum_Temperature_For_The_Last_Hour").cast(DoubleType()))
df1 = df1.withColumn("Minimum_Temperature_For_The_Last_Hour", F.col("Minimum_Temperature_For_The_Last_Hour").cast(DoubleType()))
df1 = df1.withColumn("Maximum_Relative_Humid_Temperature_For_The_Last_Hour", F.col("Maximum_Relative_Humid_Temperature_For_The_Last_Hour").cast(DoubleType()))
df1 = df1.withColumn("Minimum_Relative_Humid_Temperature_For_The_Last_Hour", F.col("Minimum_Relative_Humid_Temperature_For_The_Last_Hour").cast(DoubleType()))
df1 = df1.withColumn("Wind_speed_In_Metres_Per_Second", F.col("Wind_speed_In_Metres_Per_Second").cast(DoubleType()))




filtered_df = df1.filter((F.col("Date") >= "2009-10-21") & (F.col("Date") <= "2009-10-29"))
#filtered_df.show(5)
median_value_Atmos_Pressure = filtered_df.approxQuantile("Atmospheric_Pressure", [0.5], 0.01)[0]
median_value_Solar_Rad = filtered_df.approxQuantile("Solar_Radiation", [0.5], 0.01)[0]
median_value_Max_Temp = filtered_df.approxQuantile("Maximum_Temperature_For_The_Last_Hour", [0.5], 0.01)[0]
median_value_Min_Temp = filtered_df.approxQuantile("Minimum_Temperature_For_The_Last_Hour", [0.5], 0.01)[0]
median_value_Max_Humid = filtered_df.approxQuantile("Maximum_Relative_Humid_Temperature_For_The_Last_Hour", [0.5], 0.01)[0]
median_value_Min_Humid = filtered_df.approxQuantile("Minimum_Relative_Humid_Temperature_For_The_Last_Hour", [0.5], 0.01)[0]
median_value_Wind_Speed = filtered_df.approxQuantile("Wind_speed_In_Metres_Per_Second", [0.5], 0.01)[0]


mean_value_Precipitation = filtered_df.agg(mean("Total_Precipitation")).collect()[0][0]
print(median_value_Atmos_Pressure)
print(mean_value_Precipitation)
print(median_value_Solar_Rad)
print(median_value_Max_Temp)
print(median_value_Min_Temp)
print(median_value_Max_Humid)
print(median_value_Min_Humid)
print(median_value_Wind_Speed)



1001.3
0.0
1723.0
33.0
31.3
59.0
52.0
2.6


In [111]:
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import mean


df1 = df1.withColumn("Total_Precipitation", F.col("Total_Precipitation").cast(DoubleType()))
df1 = df1.withColumn("Atmospheric_Pressure", F.col("Atmospheric_Pressure").cast(DoubleType()))
df1 = df1.withColumn("Solar_Radiation", F.col("Solar_Radiation").cast(DoubleType()))
df1 = df1.withColumn("Maximum_Temperature_For_The_Last_Hour", F.col("Maximum_Temperature_For_The_Last_Hour").cast(DoubleType()))
df1 = df1.withColumn("Minimum_Temperature_For_The_Last_Hour", F.col("Minimum_Temperature_For_The_Last_Hour").cast(DoubleType()))
df1 = df1.withColumn("Maximum_Relative_Humid_Temperature_For_The_Last_Hour", F.col("Maximum_Relative_Humid_Temperature_For_The_Last_Hour").cast(DoubleType()))
df1 = df1.withColumn("Minimum_Relative_Humid_Temperature_For_The_Last_Hour", F.col("Minimum_Relative_Humid_Temperature_For_The_Last_Hour").cast(DoubleType()))
df1 = df1.withColumn("Wind_speed_In_Metres_Per_Second", F.col("Wind_speed_In_Metres_Per_Second").cast(DoubleType()))




filtered_df = df1.filter((F.col("Date") >= "2009-10-21") & (F.col("Date") <= "2009-10-29"))
#filtered_df.show(5)
median_value_Atmos_Pressure = filtered_df.approxQuantile("Atmospheric_Pressure", [0.5], 0.01)[0]
median_value_Solar_Rad = filtered_df.approxQuantile("Solar_Radiation", [0.5], 0.01)[0]
median_value_Max_Temp = filtered_df.approxQuantile("Maximum_Temperature_For_The_Last_Hour", [0.5], 0.01)[0]
median_value_Min_Temp = filtered_df.approxQuantile("Minimum_Temperature_For_The_Last_Hour", [0.5], 0.01)[0]
median_value_Max_Humid = filtered_df.approxQuantile("Maximum_Relative_Humid_Temperature_For_The_Last_Hour", [0.5], 0.01)[0]
median_value_Min_Humid = filtered_df.approxQuantile("Minimum_Relative_Humid_Temperature_For_The_Last_Hour", [0.5], 0.01)[0]
median_value_Wind_Speed = filtered_df.approxQuantile("Wind_speed_In_Metres_Per_Second", [0.5], 0.01)[0]


mean_value_Precipitation = filtered_df.agg(mean("Total_Precipitation")).collect()[0][0]



from pyspark.sql.functions import col, when, expr
from pyspark.sql.functions import lit
df2 = df1.filter((F.col("Date") >= "2008-01-01") & (F.col("Date") <= "2008-10-20"))
#df1 = df1.withColumn("Date", when((df1["Date"] == "2008-01-01"), "2009-01-01").otherwise(df1["Age"]))
#df1.show(5)


condition = (col("Date").startswith("2008"))

# Update the "Date" column by changing the year from 2008 to 2009
df3 = df2.withColumn("Date", when(condition, expr("to_date('2009-' || substr(Date, 6))")).otherwise(col("Date")))
#df3 = df2.withColumn("Atmospheric_Pressure", when(condition, expr("to_date('2009-' || substr(Date, 6))")).otherwise(col("Date")))
df3 = df3.withColumn("Atmospheric_Pressure", lit(median_value_Atmos_Pressure))
df3 = df3.withColumn("Solar_Radiation", lit(median_value_Solar_Rad))
df3 = df3.withColumn("Maximum_Temperature_For_The_Last_Hour", lit(median_value_Max_Temp))
df3 = df3.withColumn("Minimum_Temperature_For_The_Last_Hour", lit(median_value_Min_Temp))
df3 = df3.withColumn("Maximum_Relative_Humid_Temperature_For_The_Last_Hour", lit(median_value_Max_Humid))
df3 = df3.withColumn("Minimum_Relative_Humid_Temperature_For_The_Last_Hour", lit(median_value_Wind_Speed))
df3 = df3.withColumn("Wind_speed_In_Metres_Per_Second", lit(mean_value_Precipitation))

df3.show(20)

+----------+-----+-------------------+--------------------+---------------+-------------------------------------+-------------------------------------+----------------------------------------------------+----------------------------------------------------+-------------------------------+------+-----+------------+------------+-----------+------------+---------+
|      Date| Time|Total_Precipitation|Atmospheric_Pressure|Solar_Radiation|Maximum_Temperature_For_The_Last_Hour|Minimum_Temperature_For_The_Last_Hour|Maximum_Relative_Humid_Temperature_For_The_Last_Hour|Minimum_Relative_Humid_Temperature_For_The_Last_Hour|Wind_speed_In_Metres_Per_Second|Region|State|Station_Name|Station_Code|   Latitude|   Longitude|Elevation|
+----------+-----+-------------------+--------------------+---------------+-------------------------------------+-------------------------------------+----------------------------------------------------+----------------------------------------------------+---------------

In [112]:
df3.count()

2818

In [113]:
final_df = df1.union(df3)

# Show the resulting DataFrame
final_df.show()

+----------+-----+-------------------+--------------------+---------------+-------------------------------------+-------------------------------------+----------------------------------------------------+----------------------------------------------------+-------------------------------+------+-----+------------+------------+-----------+------------+---------+
|      Date| Time|Total_Precipitation|Atmospheric_Pressure|Solar_Radiation|Maximum_Temperature_For_The_Last_Hour|Minimum_Temperature_For_The_Last_Hour|Maximum_Relative_Humid_Temperature_For_The_Last_Hour|Minimum_Relative_Humid_Temperature_For_The_Last_Hour|Wind_speed_In_Metres_Per_Second|Region|State|Station_Name|Station_Code|   Latitude|   Longitude|Elevation|
+----------+-----+-------------------+--------------------+---------------+-------------------------------------+-------------------------------------+----------------------------------------------------+----------------------------------------------------+---------------

In [114]:
print(df1.count())
print(df3.count())
print(final_df.count())

84428
2818
87246


In [115]:
output_csv_path = "final_weather_data_north3"
df_transformed_single_partition = final_df.coalesce(1)

# Write the DataFrame to a single CSV file
df_transformed_single_partition.write.mode("overwrite").csv(output_csv_path, header=True, compression="none")

In [59]:
# df1 = df1.withColumn("Total_Precipitation", F.col("Total_Precipitation").cast(DoubleType()))
# df1.createOrReplaceTempView("my_table")

# query = f"""
#     SELECT approx_percentile(Total_Precipitation, 0.5, 0.01) as median_col1
#     FROM my_table
#     WHERE Date BETWEEN "2009-10-21" AND "2009-10-29"
# """

# result = spark.sql(query)

# # Extract the median value from the result
# median_col1 = result.collect()[0]["median_col1"]
# print(median_col1)

AnalysisException: 'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'

In [40]:
df1.printSchema()


root
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Total_Precipitation: string (nullable = true)
 |-- Atmospheric_Pressure: string (nullable = true)
 |-- Solar_Radiation: string (nullable = true)
 |-- Maximum_Temperature_For_The_Last_Hour: string (nullable = true)
 |-- Minimum_Temperature_For_The_Last_Hour: string (nullable = true)
 |-- Maximum_Relative_Humid_Temperature_For_The_Last_Hour: string (nullable = true)
 |-- Minimum_Relative_Humid_Temperature_For_The_Last_Hour: string (nullable = true)
 |-- Wind_speed_In_Metres_Per_Second: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Station_Name: string (nullable = true)
 |-- Station_Code: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Elevation: string (nullable = true)
 |-- Date1: timestamp (nullable = true)

